# HW4 2D 오브젝트 그룹 (15점)

?분반 HW4-1??그룹:

* 이름: ???
* 이름: ???
* 이름: ???

이 과제는 3 단계로 이루어진다.
각 단계를 완수할 때마다 부분점수가 주어지지만, 주의할 점은 실행가능한 코드가 아니면 0점이라는 것이다.

예를 들어 1단계까지만 했다면 1단계가 완성되어 실행가능한 형태의 코드로 제출해야 한다.

1단계를 수행한 후 2단계를 하려다 만 코드라면 점수를 얻지 못하고 0점이다.

1단와와 2단계를 완료한 직후에 파일을 따로 저장해 놓고 그 다음 단계 작업을 진행하는 것을 추천한다.

수정해도 된다고 밝힌 클래스의 정의만 수정해야 한다.

## Level 1. 일차원적 2D 오브젝트 그룹 클래스 정의 (4점)
여러개의 ShapeObj2D로 구성된 GroupObj2D의 정의를 완성하라.
즉, toString 및 Displayable과 ToSVG에서 오버라이드하라고 요구하는 메소드를 구현하라는 뜻이다.

테스트로 3개 이상의 ShapeObj2D를 함께 묶어놓은 GroupObj2D 오브젝트를 정의하여 display 해보라.

## Level 2. HW3의 변환 기능 추가 (5점)
HW3에서 작성한 3가지 변환함수 메소드를 Trans2D 인터페이스로 추상화하라.

그리고 ShapeObj2D와 GroupObj2D 클래스가 공통적인 인터페이스 Trans2D를 구현하도록 두 클래스의 정의를 수정하라.

이차원 변환에 테스트는 Level 1에서 정의했던 GroupObj2D 오브젝트에 대해 수행하면 된다.

## Level 3. 중첩 가능한 2D 오브젝트 그룹 클래스로 개선 (6점)

GroupObj2D 클래스의 멤버 변수 배열인 objects에 ShapeObj2D 인스턴스 뿐만 아니라 GroupObj2D 인스턴스도 포함될 수 있도록
GroupObj2D 클래스의 정의를 개선하라. 필요하다면 ShapeObj2D 클래스의 (심하게 뜯어고치는 건 안되고) 정의를 약간은 수정해도 된다.


In [1]:
import org.apache.commons.lang3.tuple.*;

interface Displayable {
    public void display();
}

interface ToSVG {
    public String toSVG();
    
    // points of the bounding rectangle
    public Integer minX();
    public Integer minY();
    public Integer maxX();
    public Integer maxY();
    // default methods using above four abstract methods
    default public Pair<Integer,Integer> minPoint() {
        return Pair.of( this.minX(), this.minY() );
    }
    default public Pair<Integer,Integer> maxPoint() {
        return Pair.of( this.maxX(), this.maxY() );
    }
}

In [2]:
import io.github.spencerpark.ijava.runtime.*;
import org.apache.commons.lang3.tuple.*;

abstract class Shape {
    // 인스턴스 변수
    int width;  // 양과 음의 정수값 모두 가능
    int height; // 양과 음의 정수값 모두 가능
    String fill; // 도형의 안쪽을 채우는 색깔
    double opacity; // 도형을 그렸을 때 투명도
    
    Shape(int width, int height, String fill, double opacity) {
        this.width = width;
        this.height = height;
        this.fill = fill;
        this.opacity = opacity;
    }

    // 추상 메소드 
    abstract double area(); // 넓이 계산
    abstract String toSVGshape(Pair<Integer,Integer> point); // SVG 기본 도형 태그 생성
    
    @Override
    public String toString() {
        return super.toString()
            + String.format("(width=%d, height=%d, fill=%s, opacity=%f)",
                            width, height, fill, opacity );
    }

    void display() { // 이미지 형태로 보여주기 위한 메소드
        Pair<Integer,Integer> point = Pair.of( (width<0)? Math.abs(width) :0,
                                              (height<0)? Math.abs(height):0 );
        String svgStr = String.format(
            "<svg width='%d' height='%d'>%s</svg>",
            Math.abs(width), Math.abs(height), this.toSVGshape(point) );
        Display.display(svgStr,"text/html");
    }
}

In [3]:
class RightTri extends Shape {
    RightTri(int width, int height, String fill, double opacity) {
        super(width, height, fill, opacity);
    }
    
    @Override
    double area() { return Math.abs(width * height) / 2; } // 삼각형 넓이공식에 맞게
    
    @Override
    String toSVGshape(Pair<Integer,Integer> point) {
        int x0 = point.getLeft();
        int y0 = point.getRight();
        return
            String.format("<circle cx='%d' cy='%d' r='3' fill='%s' opacity='%f' />",
                          x0,y0, fill, opacity)
            +
            String.format("<polygon points='%d,%d %d,%d %d,%d' fill='%s' opacity='%f' />",
                          x0,y0, x0+width,y0, x0,y0+height, fill, opacity );
    }
}

In [89]:
class Rectangle extends Shape {
    Rectangle(int width, int height, String fill, double opacity) {
        super(width, height, fill, opacity);
    }
    
    @Override
    double area() { return Math.abs(width * height); } // 직사각형 넓이공식에 맞게
    
    @Override
    String toSVGshape(Pair<Integer,Integer> point) {
        int x0 = point.getLeft();
        int y0 = point.getRight();
        return
            String.format("<circle cx='%d' cy='%d' r='3' fill='%s' opacity='%f' />",
                          x0,y0, fill, opacity)
            +
            String.format("<rect x='%d' y='%d' width='%d' height='%d' fill='%s' opacity='%f' />",
                          (width>0)? x0: x0+width, (height>0)? y0: y0+height,
                          Math.abs(width), Math.abs(height), fill, opacity );
    }
}

In [90]:
interface Trans2D { // 변환을 한 다음에 변환만 할거에요? 아니죠 display도 하고 그래야죠?
    InterfaceObj2D translate(int dx, int dy);
    InterfaceObj2D scale(int rx, int ry);
    InterfaceObj2D rotate(int k);
}

interface InterfaceObj2D extends Displayable, ToSVG, Trans2D { } // 인터페이스 상속

In [91]:
class ShapeObj2D implements InterfaceObj2D {
    Pair<Integer, Integer> point;
    Shape shape;
    
    ShapeObj2D(Pair<Integer, Integer> point, Shape shape) {
        this.point = point;
        this.shape = shape;
    }
    
    @Override
    public String toString() {
        return
            super.toString()
            +
            String.format("( point=%s, shape=%s )",
                          point.toString(), shape.toString() );
    }
    
    // Displayable에서 오버라이드하라고 요구하는 메소드를 구현
    @Override // 지금은 무조건 원점부터 1사분면만 그리고 있다고 생각하고 작성하고 있음
    public void display() { // 이미지 형태로 보여주기 위한 메소드
        String svgStr = String.format(
            "<svg width='%d' height='%d'>%s</svg>",
            maxX(), maxY(), this.toSVG() );
        Display.display(svgStr,"text/html");
    }
    
    // ToSVG에서 오버라이드하라고 요구하는 메소드들을 구현
    @Override
    public String toSVG() { return shape.toSVGshape(point); }    
    // points of the bounding rectangle
    @Override
    public Integer minX() { return Math.min(point.getLeft(), point.getLeft()+shape.width); }
    @Override
    public Integer minY() { return Math.min(point.getRight(), point.getRight()+shape.height); }
    @Override
    public Integer maxX() { return Math.max(point.getLeft(), point.getLeft()+shape.width); }
    @Override
    public Integer maxY() { return Math.max(point.getRight(), point.getRight()+shape.height); }

    // Trans2D에서 오버라이드하라고 요구하는 메소드들을 구현 (실제 구현은 생략 HW3과 대략 마찬가지)
    @Override
    public InterfaceObj2D translate(int dx, int dy) {
        // this.point = Pair.of(point.getLeft()+dx,point.getRight()+dy); // 원본 좌표 변경
        // return this;
        return new ShapeObj2D(Pair.of(point.getLeft()+dx,point.getRight()+dy), shape);
    }
    @Override
    public InterfaceObj2D scale(int rx, int ry) { return null; } // 나머지도 HW3과 마찬가지로
    @Override
    public InterfaceObj2D rotate(int k)  { return null; } // 나머지도 HW3과 마찬가지로
}

In [92]:
class GroupObj2D implements InterfaceObj2D {
    InterfaceObj2D [] objects;
    
    GroupObj2D(InterfaceObj2D[] objects) { // ShapeObj2D의 배열을 받는 생성자
        this.objects = objects;
    }
    
    @Override
    public String toString() { // 반복문으로 String으로 만들어 줍시다
        String str = "";
        for (var o : objects) {
            str += "\n" + o.toString();
        }
        return "GroupObj2D: " + str;
    }
    
    // Displayable에서 오버라이드하라고 요구하는 메소드를 구현
    @Override // 지금은 무조건 원점부터 1사분면만 그리고 있다고 생각하고 작성하고 있음
    public void display() { // 이미지 형태로 보여주기 위한 메소드
        String svgStr = String.format(
            "<svg width='%d' height='%d'>%s</svg>",
            maxX(), maxY(), this.toSVG() );
        Display.display(svgStr,"text/html");
    }

    // ToSVG에서 오버라이드하라고 요구하는 메소드들을 구현
    @Override 
    public String toSVG() { // 각각의 ShapeObj2D의 toSVG의 결과를 이어붙이면
        String str = "";
        for (var o : objects) {
            str += "\n" + o.toSVG();
        }
        return str;
    } 
    // points of the bounding rectangle
    @Override
    public Integer minX() {
        // objects 에 아무것도 안 들어 있는 경우를 혹시 대비해서 (원래 이런 그룹은 만들며 안됨)
        if (objects.length == 0) return 0;
        
        int x = objects[0].minX();
        for (var o : objects)
            x = Math.min(x, o.minX());
        return x;
    }
    @Override
    public Integer minY() {
        // objects 에 아무것도 안 들어 있는 경우를 혹시 대비해서 (원래 이런 그룹은 만들며 안됨)
        if (objects.length == 0) return 0;
        
        int y = objects[0].minY();
        for (var o : objects)
            y = Math.min(y, o.minY());
        return y;        
    }
    @Override
    public Integer maxX() {
        // objects 에 아무것도 안 들어 있는 경우를 혹시 대비해서 (원래 이런 그룹은 만들며 안됨)
        if (objects.length == 0) return 0;
        
        int x = objects[0].maxX();
        for (var o : objects)
            x = Math.max(x, o.maxX());
        return x;        
    }
    @Override
    public Integer maxY() {
        // objects 에 아무것도 안 들어 있는 경우를 혹시 대비해서 (원래 이런 그룹은 만들며 안됨)
        if (objects.length == 0) return 0;
        
        int y = objects[0].maxY();
        for (var o : objects)
            y = Math.max(y, o.maxY());
        return y; 
    }
    
    // Trans2D에서 오버라이드하라고 요구하는 메소드들을 구현
    @Override
    public InterfaceObj2D translate(int dx, int dy) {
        InterfaceObj2D objs[] = new InterfaceObj2D[objects.length];
        for (int i=0; i<objs.length; ++i) 
            objs[i] = objects[i].translate(dx,dy);
        return new GroupObj2D( objs ); // 대략은 맞는데
    }
    @Override
    public InterfaceObj2D scale(int rx, int ry) { return null; } // 나머지도 HW3과 마찬가지로
    @Override
    public InterfaceObj2D rotate(int k)  { return null; } // 나머지도 HW3과 마찬가지로
}

In [93]:
RightTri t1 = new RightTri(20,40,"red",0.3);
RightTri t2 = new RightTri(40,20,"blue",0.3);
Rectangle r1 = new Rectangle(30,80,"green",0.3);

ShapeObj2D o1 = new ShapeObj2D(Pair.of(1,1), t1);
ShapeObj2D o2 = new ShapeObj2D(Pair.of(30,20), t2);
ShapeObj2D o3 = new ShapeObj2D(Pair.of(50,30), r1);

In [94]:
o1

REPL.$JShell$20E$ShapeObj2D@956cb03( point=(1,1), shape=REPL.$JShell$17$RightTri@6a5ad4bb(width=20, height=40, fill=red, opacity=0.300000) )

In [95]:
o2

REPL.$JShell$20E$ShapeObj2D@5d5aac9a( point=(30,20), shape=REPL.$JShell$17$RightTri@5b8c3892(width=40, height=20, fill=blue, opacity=0.300000) )

In [96]:
o3

REPL.$JShell$20E$ShapeObj2D@6fc7b695( point=(50,30), shape=REPL.$JShell$18$Rectangle@6ee6a761(width=30, height=80, fill=green, opacity=0.300000) )

In [97]:
o1.display()

<svg width='21' height='41'><circle cx='1' cy='1' r='3' fill='red' opacity='0.300000' /><polygon points='1,1 21,1 1,41' fill='red' opacity='0.300000' /></svg>

In [98]:
o2.display();

<svg width='70' height='40'><circle cx='30' cy='20' r='3' fill='blue' opacity='0.300000' /><polygon points='30,20 70,20 30,40' fill='blue' opacity='0.300000' /></svg>

In [99]:
o3.display();

<svg width='80' height='110'><circle cx='50' cy='30' r='3' fill='green' opacity='0.300000' /><rect x='50' y='30' width='30' height='80' fill='green' opacity='0.300000' /></svg>

In [100]:
ShapeObj2D objs[] = { o1, o2, o3 };

GroupObj2D group1 = new GroupObj2D(objs);

In [101]:
group1.display()

<svg width='80' height='110'>
<circle cx='1' cy='1' r='3' fill='red' opacity='0.300000' /><polygon points='1,1 21,1 1,41' fill='red' opacity='0.300000' />
<circle cx='30' cy='20' r='3' fill='blue' opacity='0.300000' /><polygon points='30,20 70,20 30,40' fill='blue' opacity='0.300000' />
<circle cx='50' cy='30' r='3' fill='green' opacity='0.300000' /><rect x='50' y='30' width='30' height='80' fill='green' opacity='0.300000' /></svg>

In [102]:
GroupObj2D group2 = (GroupObj2D) group1.translate(100,50);

In [103]:
group1.display();

<svg width='80' height='110'>
<circle cx='1' cy='1' r='3' fill='red' opacity='0.300000' /><polygon points='1,1 21,1 1,41' fill='red' opacity='0.300000' />
<circle cx='30' cy='20' r='3' fill='blue' opacity='0.300000' /><polygon points='30,20 70,20 30,40' fill='blue' opacity='0.300000' />
<circle cx='50' cy='30' r='3' fill='green' opacity='0.300000' /><rect x='50' y='30' width='30' height='80' fill='green' opacity='0.300000' /></svg>

In [104]:
group2.display();

<svg width='180' height='160'>
<circle cx='101' cy='51' r='3' fill='red' opacity='0.300000' /><polygon points='101,51 121,51 101,91' fill='red' opacity='0.300000' />
<circle cx='130' cy='70' r='3' fill='blue' opacity='0.300000' /><polygon points='130,70 170,70 130,90' fill='blue' opacity='0.300000' />
<circle cx='150' cy='80' r='3' fill='green' opacity='0.300000' /><rect x='150' y='80' width='30' height='80' fill='green' opacity='0.300000' /></svg>

In [105]:
InterfaceObj2D objs3[] = { o1, o2, group2 };

GroupObj2D group3 = new GroupObj2D(objs3);

In [106]:
group3.display()

<svg width='180' height='160'>
<circle cx='1' cy='1' r='3' fill='red' opacity='0.300000' /><polygon points='1,1 21,1 1,41' fill='red' opacity='0.300000' />
<circle cx='30' cy='20' r='3' fill='blue' opacity='0.300000' /><polygon points='30,20 70,20 30,40' fill='blue' opacity='0.300000' />

<circle cx='101' cy='51' r='3' fill='red' opacity='0.300000' /><polygon points='101,51 121,51 101,91' fill='red' opacity='0.300000' />
<circle cx='130' cy='70' r='3' fill='blue' opacity='0.300000' /><polygon points='130,70 170,70 130,90' fill='blue' opacity='0.300000' />
<circle cx='150' cy='80' r='3' fill='green' opacity='0.300000' /><rect x='150' y='80' width='30' height='80' fill='green' opacity='0.300000' /></svg>

In [19]:
o1.toSVG() + o2.toSVG() + o3.toSVG()

<circle cx='101' cy='51' r='3' fill='red' opacity='0.300000' /><polygon points='101,51 121,51 101,91' fill='red' opacity='0.300000' /><circle cx='130' cy='70' r='3' fill='blue' opacity='0.300000' /><polygon points='130,70 170,70 130,90' fill='blue' opacity='0.300000' /><circle cx='150' cy='80' r='3' fill='green' opacity='0.300000' /><rect x='150' y='80' width='30' height='80' fill='green' opacity='0.300000' />

In [20]:
group1.minX()

101

In [21]:
group1.minY()

51

In [22]:
group1.maxX()

180

In [23]:
group1.maxY()

160

In [24]:
group1.translate(100,200);

GroupObj2D: 
REPL.$JShell$20$ShapeObj2D@6ee98dbe( point=(201,251), shape=REPL.$JShell$17$RightTri@1568d424(width=20, height=40, fill=red, opacity=0.300000) )
REPL.$JShell$20$ShapeObj2D@62778677( point=(230,270), shape=REPL.$JShell$17$RightTri@56fb44ec(width=40, height=20, fill=blue, opacity=0.300000) )
REPL.$JShell$20$ShapeObj2D@21310ef6( point=(250,280), shape=REPL.$JShell$18$Rectangle@78dc9254(width=30, height=80, fill=green, opacity=0.300000) )

In [41]:
Integer arr1[] = {1,2,3};
Integer arr2[] = arr1.clone();

In [42]:
Arrays.toString(arr2)

[1, 2, 3]